In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets==2.13.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import sys
import subprocess
import torch
import argparse
from random import randrange
from torch.utils.data import IterableDataset
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,  # For loading pre-trained causal language models
    AutoTokenizer,  # For tokenizing text inputs
    BitsAndBytesConfig,  # Configuration for 8-bit optimizations
    HfArgumentParser,  # For parsing command-line arguments
    TrainingArguments,  # Configuration for model training
    pipeline,  # For creating inference pipelines
    logging,  # For configuring logging
)
from peft import (
    LoraConfig,  # Configuration for LoRA (Low-Rank Adaptation)
    prepare_model_for_kbit_training,  # Prepare model for k-bit (low-bit) training
    PeftModel,  # Get a PEFT (Parameter-Efficient Fine-Tuning) model
    AutoPeftModelForCausalLM,  # Auto class for PEFT models
)
from accelerate import Accelerator  # For hardware acceleration
from trl import SFTTrainer  # For supervised fine-tuning with TRL

In [ ]:
HUGGINGFACE_TOKEN=hf_YourHuggingFaceToken


In [ ]:
import os
import subprocess
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the Hugging Face token from environment variable
token = os.getenv("HUGGINGFACE_TOKEN")

if token is None:
    raise ValueError("Hugging Face token not found in environment variables")

# Set up Git credential helper
subprocess.run(["git", "config", "--global", "credential.helper", "store"])

# Run the Hugging Face CLI login command with the token and add to git credential
subprocess.run(["huggingface-cli", "login", "--token", token, "--add-to-git-credential"])


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Define model name and Bitsandbytes configuration
model_name = "meta-llama/Llama-2-7b-hf"
use_4bit = True
bnb_4bit_compute_dtype = "float16"
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_4bit_quant_type = "nf4"

# Bitsandbytes int-4 configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype
)

# Define a function to load the model and tokenizer
def load_model(model_name, bnb_config):
    device_map = {"": 0}
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        use_cache=False,
        use_auth_token=True,
        device_map=device_map
    )
    model.config.pretraining_tp = 1
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    return model, tokenizer

# Load the model and tokenizer
model, tokenizer = load_model(model_name, bnb_config)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load the Swahili news classification dataset
dataset = load_dataset('derekiya/swahili_news_categorize', split='train')


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/49334 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5482 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/derekiya___parquet/derekiya--swahili_news_categorize-63d7d545eb492041/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=1,
    fp16=False,
    bf16=False,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    learning_rate=2e-4,
    weight_decay=0.001,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    save_steps=0,
    logging_steps=25
)


In [ ]:
from trl import SFTTrainer

# Set supervised fine-tuning parameters
max_seq_length = None
packing = False

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=max_seq_length,
    packing=packing,
    dataset_text_field="instructions-text"
)



Map:   0%|          | 0/49334 [00:00<?, ? examples/s]

In [ ]:

# Start training
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.755300
50,2.560200
75,2.418800
100,2.205500
125,2.262800
150,2.146600
175,2.217200
200,2.004400
225,2.124900
250,1.977900


In [ ]:
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

# Define the fine-tuned model name and output directories
finetuned_model = "derekiya/swahili_news_classification"


# Define the fine-tuned model name and output directories
output_dir = "./checkpoints"

# Save trained model
trainer.model.save_pretrained(output_dir)
base_model = AutoModelForCausalLM.from_pretrained(model_name)

model = PeftModel.from_pretrained(base_model, output_dir)
model = model.merge_and_unload()

# Merge the LoRA layers with the base model

model.save_pretrained(output_dir, safe_serialization=True)

# Save tokenizer for easy inference
tokenizer.save_pretrained(output_dir)

# Push fine-tuned model and tokenizer to Hugging Face Hub
model.push_to_hub(finetuned_model, use_auth_token=True)
tokenizer.push_to_hub(finetuned_model, use_auth_token=True)





#Test

In [3]:
!pip install accelerate




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model and tokenizer from Hugging Face Hub
model_name = "derekiya/swahili_news_classification"

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [2]:
# Define a function to classify news articles using the fine-tuned model
def classify_news_article(instruction, input_text):
    prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Categorize the news article into one of the 6 categories: UCHUMI, KITAIFA, MICHEZO, KIMATAIFA, BURUDANI, AFYA

### Input:
{input_text}

### Response:
"""
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

    # Generate response
    outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=0.5)

    # Decode and return the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the classification response by removing the prompt from the generated text
    classification_response = generated_text.replace(prompt, '').strip().split('\n')[0]
    return classification_response

In [4]:
#Example usage
instruction = "Categorize the news article into one of the 6 categories: UCHUMI, KITAIFA, MICHEZO, KIMATAIFA, BURUDANI, AFYA"
input_text = "RABAT, MOROCCO UFALME wa Morocco umelaani vikali vitendo vya Mwenyekiti wa Tume ya Umoja wa Afrika Nkosazana Dlamini-Zuma, unayeamini anahujumu maombi ya taifa hilo kurejea katika umoja huo. Mfalme Mohammed VI alitangaza rasmi nia ya Morocco kujiunga tena Umoja wa Afrika (AU) wakati akitoa hotuba nchini Rwanda Julai 17 mwaka huu. Nchi hiyo ilijitoa kutoka AU miaka 32 iliyopita kupinga kitendo cha chombo hicho kutambua harakati za uhuru wa Sahara Magharibi zinazopiganiwa na vuguvugu la Polisario. Morocco inasisitiza eneo hilo ni sehemu ya himaya yake. Ni miezi zaidi ya minne sasa tangu hotuba ya mfalme nchini Rwanda, na ijapokuwa mapema mwezi uliopita Dlamini-Zuma alimuahidi Waziri wa Mambo ya Nje wa Morocco, Salaheddine Mezouar kuwasilisha ombi hilo kwa nchi nyingine za AU, hakuna maendeleo yaliyopigwa hadi sasa. Wizara ya Mambo ya Nje na Uhusiano wa Kimataifa ilitoa taarifa kwamba, “baada ya ucheleweshaji usio na msingi kwa kutosambaza ombi la Morocco kwa wanachama wa AU, Nkosazana Dlamini-Zuma anaendelea kutuhujumu kwa kuweka masharti mapya kinyume na yale yanayofahamika ili ombi letu lisipate baraka kutoka wanachama wa AU.” “Mwenyekiti wa Tume ya AU anashangaza kwa kushindwa kufanya kazi bila kuegemea upande na kufuata sheria na kanuni za AU na matakwa ya nchi wanachama wizara hiyo iliongeza. Takriban nchi 36 kati ya 54 wanachama wa AU haziitambui Sahara Magharibi kama eneo huru na ni wakati wa kufuta uamuzi huo ilisema taarifa hiyo."
response = classify_news_article(instruction, input_text)
print(f"Generated Response:\n{response}")


Generated Response:
KIMATAIFA
